<a href="https://colab.research.google.com/github/zanzivyr/Optimizers/blob/main/LP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Linear Programming

Today we will continue our study of optimization with a brief overview of Linear Programming.

## Resources

These are resources I used to learn about Linear Programming:

- The Organic Chemistry Tutor - https://youtu.be/Bzzqx1F23a8
- Meghan de Witt - https://youtu.be/rzRZLGD_aeE
- Wikipedia - https://en.wikipedia.org/wiki/Linear_programming
- Timeparticle - https://www.youtube.com/watch?v=5bxsxM2UTb4
- Geeks for Geeks - https://www.geeksforgeeks.org/singular-value-decomposition-svd/
- Section.io - https://www.section.io/engineering-education/moore-penrose-pseudoinverse/
- Jon Krohn - https://youtu.be/vXk-o3PVUdU

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

## Instantiate

First we'll create a LP with an objective function and 2 constraint inequalities as a matrix.

We can use random integer values for the coefficients.

The A matrix shall be constructed as:

1. **Row 1** = coefficients of the **objective**
2. **Row 2+** = coefficients of the **constraints**
3. **Column vector b** = solutions to the objective and constraints

### Augmentations

We're also going to augment the objective and constraints.

By adding "slack" variables to the constraints (one per constraint) we can convert the inequality to an equality. (Basically, by having another term, we can guarantee that the equation will become equal... it's a stop gap) Then when checking that the solutions to the variables meet the final constraint, we can say if our solution is feasible, or unfeasible.

**Note**: The objective row should have zeroes for the slack variables.

This is called the Simplex Method

In [ ]:
A = tf.random.normal(
    shape=[3,2],
    mean=0,
    stddev=1,
    dtype=tf.dtypes.float32,
    seed=42,
    name=None
)
eye = tf.constant([[0,0],[1,0],[0,1]], dtype=tf.dtypes.float32)
b = tf.random.normal(
    shape=[3,1],
    mean=0,
    stddev=1,
    dtype=tf.dtypes.float32,
    seed=42,
    name=None
)
A = tf.concat([A, eye], 1)
A, b

(<tf.Tensor: shape=(3, 4), dtype=float32, numpy=
 array([[-0.41014114,  0.02199026,  0.        ,  0.        ],
        [-0.8885235 , -0.8977638 ,  1.        ,  0.        ],
        [-1.8565961 ,  1.370516  ,  0.        ,  1.        ]],
       dtype=float32)>, <tf.Tensor: shape=(3, 1), dtype=float32, numpy=
 array([[-0.99965364],
        [-1.4047071 ],
        [-0.941157  ]], dtype=float32)>)

## Singular Value Decomposition (SVD)

Use the Singular Value Decomposition to find the Moore-Penrose Pseudoinverse

In [ ]:
# SVD
u, s, vt = np.linalg.svd(A, full_matrices=True)
u, s, vt

(array([[ 0.1299178 ,  0.09825184,  0.9866448 ],
        [ 0.11778145,  0.98650354, -0.11374681],
        [ 0.9845044 , -0.13098618, -0.11659212]], dtype=float32),
 array([2.5455608 , 1.6044058 , 0.18819794], dtype=float32),
 array([[-0.7800887 ,  0.4896351 ,  0.04626935,  0.38675344],
        [-0.419869  , -0.66255385,  0.61487156, -0.08164155],
        [-0.46298283, -0.1911659 , -0.60439986, -0.61951864],
        [-0.02861   , -0.5336061 , -0.50447285,  0.67819846]],
       dtype=float32))

### SVD Inverse

Using the SVD, we can now invert the matrix

In [ ]:
# Moore-Penrose Pseudo Inverse
vtt = np.transpose(vt)
sinv = np.linalg.inv(np.diag(s))
ut = np.transpose(u)

vtt.shape, sinv.shape, ut.shape

((4, 4), (3, 3), (3, 3))

Σ transpose needs to be the same dimensions as A.

We can concatenate on another column if A is overdetermined or row if A is underdetermined.

In [ ]:
sinv = np.concatenate((sinv, np.array([[0,0,0]])), axis=0)
sinv.shape

(4, 3)

## Moore-Penrose Pseudoinverse

Now we can find A dagger:

A† = VTᵀ Σ Uᵀ

In [ ]:
Adag = np.dot(vtt, np.dot(sinv, ut))
Adag

array([[-2.49275517, -0.01443299,  0.01940323],
       [-1.01778902, -0.26918982,  0.36189082],
       [-3.12860567,  0.7455071 ,  0.3421327 ],
       [-3.23314356,  0.34213272,  0.54004687]])

A† is the pseudoinverse. 

Now we can solve the linear equation for x:

AA† x = A† b

In [ ]:
x = np.dot(Adag, b)
x

array([[2.4939044 ],
       [1.05497328],
       [1.75830236],
       [2.24315858]])

## Optimizing the objective

Since we have values for x1 and x2, we can now substitute these values into the objective and receive the optimized cost.

In [ ]:
cost = np.dot(A[0], x)[0]

print("Cost: " + str(cost) + ", when")
i = 1
for v in x:
  print("x" + str(i) + " = " + str(v[0]))
  i+=1

Cost: -0.999653656525371, when
x1 = 2.4939044026875252
x2 = 1.054973279923544
x3 = 1.7583023608241501
x4 = 2.2431585834331664


### Check against final constraints

At this point it is necessary to check against the final constraints to ensure the solution is feasible.

Usually this will look like x1, x2 ≥ 0

With the Simplex method, we also need to add the additional slack variables such that:

x1, x2, u, v, ... ≥ 0

In [ ]:
flag = False
for v in range(1, len(x)):
  if(x[v] < 0):
    flag = True
    print("The solution is infeasible.")
    break

if not flag:
  print("Congrats! The solution is feasible!")

Congrats! The solution is feasible!
